# 1. 環境設定 (Windows 版本)


In [ ]:
import sys
print(f"Python 版本: {sys.version}")
!{sys.executable} -m pip install -U numpy==1.26.4

In [ ]:
import sys
!{sys.executable} -m pip install -U ray==2.6.0
import ray
print(f"Ray 版本: {ray.__version__}")

In [ ]:
import torch
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"GPU 裝置: {torch.cuda.get_device_name(0)}")

In [ ]:
import sys
!{sys.executable} -m pip install -U monai==1.2.0
import monai
print(f"MONAI 版本: {monai.__version__}")

In [ ]:
!pip install -U ml_collections
!pip install timm
!pip install einops
!pip install scipy
!pip install nibabel

In [ ]:
!git clone https://github.com/kairaun/Sam.git

In [ ]:
# 安裝 Sam
%cd Sam
!dir
!type pyproject.toml
!pip install -e .
%cd ..

In [ ]:
import sys
!{sys.executable} -m pip install -U --no-deps monailabel
import monailabel
print(f"MONAILabel 版本: {monailabel.__version__}")

In [ ]:
!pip install "pydantic<2.0"

In [ ]:
!git clone https://github.com/kairaun/CardiacSegV2.git

In [ ]:
# 安裝環境 (Windows 版本)
!pip install gdown==4.6.0

# 注意：不直接安裝 requirements.txt，因為它包含舊版本的 PyTorch
# 我們會手動安裝需要的套件
print("正在安裝其他必要套件...")

# 讀取 requirements.txt 並過濾掉 PyTorch 相關套件
import os
req_file = os.path.join('CardiacSegV2', 'requirements.txt')
if os.path.exists(req_file):
    with open(req_file, 'r') as f:
        packages = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    
    # 過濾掉已安裝或不相容的套件
    skip_packages = ['torch', 'torchvision', 'torchaudio', 'monai', 'monailabel']
    packages_to_install = [p for p in packages if not any(skip in p.lower() for skip in skip_packages)]
    
    print(f"將安裝以下套件: {packages_to_install}")
    for pkg in packages_to_install:
        try:
            print(f"安裝 {pkg}...")
            get_ipython().system(f'pip install {pkg}')
        except Exception as e:
            print(f"警告: {pkg} 安裝失敗 - {e}")
else:
    print("找不到 requirements.txt，跳過套件安裝")

In [ ]:
# 修改程式碼以適應 Windows 路徑
import os

# 取得當前工作目錄
workspace_dir = os.path.abspath('CardiacSegV2')
print(f"工作目錄: {workspace_dir}")

# 讀取並修改 setup_dir.py
setup_dir_path = os.path.join(workspace_dir, 'setup_dir.py')
with open(setup_dir_path, 'r', encoding='utf-8') as f:
    content = f.read()

# 替換路徑 (使用 os.path.join 以支援 Windows)
content = content.replace('/content/CardiacSegV2/', workspace_dir.replace('\\', '/') + '/')

with open(setup_dir_path, 'w', encoding='utf-8') as f:
    f.write(content)

print("setup_dir.py 已更新")

In [ ]:
# 修改 infer.py 和 tune.py
import os

workspace_dir = os.path.abspath('CardiacSegV2')

# 修改 infer.py
infer_path = os.path.join(workspace_dir, 'expers', 'infer.py')
if os.path.exists(infer_path):
    with open(infer_path, 'r', encoding='utf-8') as f:
        content = f.read()
    content = content.replace('sys.path.append("/content/CardiacSegV2")', 
                             f'sys.path.append("{workspace_dir.replace(chr(92), chr(92)+chr(92))}")')
    with open(infer_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print("infer.py 已更新")

# 修改 tune.py
tune_path = os.path.join(workspace_dir, 'expers', 'tune.py')
if os.path.exists(tune_path):
    with open(tune_path, 'r', encoding='utf-8') as f:
        content = f.read()
    content = content.replace('sys.path.append("/content/CardiacSegV2")', 
                             f'sys.path.append("{workspace_dir.replace(chr(92), chr(92)+chr(92))}")')
    content = content.replace('ray.init(runtime_env={"working_dir": "/content/CardiacSegV2"})',
                             f'ray.init(runtime_env={{"working_dir": "{workspace_dir.replace(chr(92), chr(92)+chr(92))}"}})') 
    with open(tune_path, 'w', encoding='utf-8') as f:
        f.write(content)
    print("tune.py 已更新")

In [ ]:
!python CardiacSegV2\setup_dir.py

# 2. 設定訓練參數


In [1]:
import os

# 使用 Windows 風格的路徑
workspace_dir = os.path.abspath('CardiacSegV2')
model_name = 'DynUNet'
data_name = 'chgh'
exp_name = 'AICUP_training'
data_dict_file_name = 'AICUP_training.json'

# 設定實驗目錄
root_exp_dir = os.path.join(workspace_dir, 'exps', 'exps', model_name, data_name, 'tune_results')

# 設定資料目錄
root_data_dir = os.path.join(workspace_dir, 'dataset', data_name)
data_dir = root_data_dir

# data dict json 路徑
data_dicts_json = os.path.join(workspace_dir, 'exps', 'data_dicts', data_name, data_dict_file_name)

# 設定模型、日誌、評估目錄
model_dir = os.path.join('.', 'models')
log_dir = os.path.join('.', 'logs')
eval_dir = os.path.join('.', 'evals')

os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(eval_dir, exist_ok=True)

# 模型路徑
best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

# 建立根實驗目錄
os.makedirs(root_exp_dir, exist_ok=True)

print(f"工作目錄: {workspace_dir}")
print(f"實驗目錄: {root_exp_dir}")
print(f"資料目錄: {data_dir}")
print(f"模型目錄: {model_dir}")

工作目錄: d:\Felix\ML\Code\CardiacSegV2
實驗目錄: d:\Felix\ML\Code\CardiacSegV2\exps\exps\DynUNet\chgh\tune_results
資料目錄: d:\Felix\ML\Code\CardiacSegV2\dataset\chgh
模型目錄: .\models


# 3. 正式訓練

執行模型訓練，訓練參數可以根據需要調整。

cd D:\Felix\ML\Code\CardiacSegV2

python expers/tune.py --tune_mode=train --exp_name=AICUP_training --data_name=chgh --data_dir=D:\Felix\ML\Code\CardiacSegV2\dataset\chgh --root_exp_dir=D:\Felix\ML\Code\CardiacSegV2\exps\exps\unet3d\chgh\tune_results --model_name=unet3d --model_dir=.\models --log_dir=.\logs --eval_dir=.\evals --start_epoch=0 --val_every=5 --max_early_stop_count=2 --max_epoch=20 --data_dicts_json=D:\Felix\ML\Code\CardiacSegV2\exps\data_dicts\chgh\AICUP_training.json --pin_memory --out_channels=4 --patch_size=4 --feature_size=48 --drop_rate=0.1 --depths 3 3 9 3 --kernel_size 7 --exp_rate 4 --norm_name=layer --a_min=-42 --a_max=423 --space_x=0.7 --space_y=0.7 --space_z=1.0 --roi_x=128 --roi_y=128 --roi_z=120 --optim=AdamW --lr=5e-4 --weight_decay=5e-4 --checkpoint=.\models\final_model.pth --use_init_weights --infer_post_process

In [3]:
# 訓練模型 (Windows 版本)
import os
import sys
import subprocess

# 確保在正確的目錄
original_dir = os.getcwd()
os.chdir(workspace_dir)

print(f"當前工作目錄: {os.getcwd()}")
print("開始訓練...\n")

# 建構訓練命令
train_cmd = [
    sys.executable, "expers/tune.py",
    "--tune_mode=train",
    f"--exp_name={exp_name}",
    f"--data_name={data_name}",
    f"--data_dir={data_dir}",
    f"--root_exp_dir={root_exp_dir}",
    f"--model_name={model_name}",
    f"--model_dir={model_dir}",
    f"--log_dir={log_dir}",
    f"--eval_dir={eval_dir}",
    "--start_epoch=0",
    "--val_every=5",
    "--max_early_stop_count=2",
    "--max_epoch=20",
    f"--data_dicts_json={data_dicts_json}",
    "--pin_memory",
    "--out_channels=4",
    "--patch_size=4",
    "--feature_size=48",
    "--drop_rate=0.1",
    "--depths", "3", "3", "9", "3",
    "--kernel_size", "7",
    "--exp_rate", "4",
    "--norm_name=layer",
    "--a_min=-42",
    "--a_max=423",
    "--space_x=0.7",
    "--space_y=0.7",
    "--space_z=1.0",
    "--roi_x=128",
    "--roi_y=128",
    "--roi_z=128",  # 修改為 128 (2的冪次方) 以避免尺寸不匹配錯誤
    "--optim=AdamW",
    "--lr=5e-4",
    "--weight_decay=5e-4",
    f"--checkpoint={final_checkpoint}",
    "--use_init_weights",
    "--infer_post_process"
]

print("執行命令:")
print(" ".join(train_cmd))
print("\n" + "="*80 + "\n")

# 使用 Popen 來即時顯示輸出
process = subprocess.Popen(
    train_cmd,
    cwd=workspace_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

# 即時打印輸出
for line in process.stdout:
    print(line, end='')

# 等待程序完成
return_code = process.wait()

# 切回原始目錄
os.chdir(original_dir)

print("\n" + "="*80)
if return_code == 0:
    print("\n✅ 訓練完成！")
    print(f"最佳模型已儲存至: {best_checkpoint}")
    print(f"最終模型已儲存至: {final_checkpoint}")
else:
    print(f"\n❌ 訓練失敗，錯誤碼: {return_code}")
    print("請查看上方的錯誤訊息以了解失敗原因")

當前工作目錄: d:\Felix\ML\Code\CardiacSegV2
開始訓練...

執行命令:
d:\Felix\ML\Code\.conda\python.exe expers/tune.py --tune_mode=train --exp_name=AICUP_training --data_name=chgh --data_dir=d:\Felix\ML\Code\CardiacSegV2\dataset\chgh --root_exp_dir=d:\Felix\ML\Code\CardiacSegV2\exps\exps\DynUNet\chgh\tune_results --model_name=DynUNet --model_dir=.\models --log_dir=.\logs --eval_dir=.\evals --start_epoch=0 --val_every=5 --max_early_stop_count=2 --max_epoch=20 --data_dicts_json=d:\Felix\ML\Code\CardiacSegV2\exps\data_dicts\chgh\AICUP_training.json --pin_memory --out_channels=4 --patch_size=4 --feature_size=48 --drop_rate=0.1 --depths 3 3 9 3 --kernel_size 7 --exp_rate 4 --norm_name=layer --a_min=-42 --a_max=423 --space_x=0.7 --space_y=0.7 --space_z=1.0 --roi_x=128 --roi_y=128 --roi_z=128 --optim=AdamW --lr=5e-4 --weight_decay=5e-4 --checkpoint=.\models\final_model.pth --use_init_weights --infer_post_process


Importing from timm.models.layers is deprecated, please import via timm.layers
Importing from t

# 4. 測試訓練結果

使用訓練好的模型進行測試評估。

In [ ]:
# 測試模型 (Windows 版本)
import os
import sys
import subprocess

# 確保在正確的目錄
original_dir = os.getcwd()
os.chdir(workspace_dir)

print(f"當前工作目錄: {os.getcwd()}")
print("開始測試...\n")

# 建構測試命令
test_cmd = [
    sys.executable, "expers/tune.py",
    "--tune_mode=test",
    f"--exp_name={exp_name}",
    f"--data_name={data_name}",
    f"--data_dir={data_dir}",
    f"--root_exp_dir={root_exp_dir}",
    f"--model_name={model_name}",
    f"--model_dir={model_dir}",
    f"--log_dir={log_dir}",
    f"--eval_dir={eval_dir}",
    f"--data_dicts_json={data_dicts_json}",
    "--pin_memory",
    "--out_channels=4",
    "--patch_size=4",
    "--feature_size=48",
    "--drop_rate=0.1",
    "--depths", "3", "3", "9", "3",
    "--kernel_size", "7",
    "--exp_rate", "4",
    "--norm_name=layer",
    "--a_min=-42",
    "--a_max=423",
    "--space_x=0.7",
    "--space_y=0.7",
    "--space_z=1.0",
    "--roi_x=128",
    "--roi_y=128",
    "--roi_z=128",
    "--optim=AdamW",
    "--lr=5e-4",
    "--weight_decay=5e-4",
    f"--checkpoint={final_checkpoint}",
    "--use_init_weights",
    "--infer_post_process",
    "--resume_tuner",
    "--save_eval_csv",
    "--test_mode"
]

print("執行命令:")
print(" ".join(test_cmd))
print("\n" + "="*80 + "\n")

# 使用 Popen 來即時顯示輸出
process = subprocess.Popen(
    test_cmd,
    cwd=workspace_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

# 即時打印輸出
for line in process.stdout:
    print(line, end='')

# 等待程序完成
return_code = process.wait()

# 切回原始目錄
os.chdir(original_dir)

print("\n" + "="*80)
if return_code == 0:
    print("\n✅ 測試完成！")
else:
    print(f"\n❌ 測試失敗，錯誤碼: {return_code}")
    print("請查看上方的錯誤訊息以了解失敗原因")